# INPUT FILES FORMATTING

In [19]:

import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder

## Replacing Portuguese Categories with English Translations

In [20]:
products_df = pd.read_csv("./data/olist_products_dataset.csv")
translations_df = pd.read_csv("./data/product_category_name_translation.csv")

translations_dict = dict(zip(translations_df["product_category_name"], translations_df["product_category_name_english"]))
products_df["product_category_name"] = products_df["product_category_name"].apply(lambda x: translations_dict[x] if x in translations_dict else x)

products_df.to_csv("./data/olist_products_dataset.csv", index=False)

## Removing All Double Quotes

In [21]:
def remove_double_quotes_from_file(file_path):
    try:
        df = pd.read_csv(file_path)
        df = df.applymap(lambda x: str(x).replace('"', ''))
        df.to_csv(file_path, index=False)
        print(f"Double quotes removed from {file_path}.")
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

data_folder = "data/"
files = [os.path.join(data_folder, file) for file in os.listdir(data_folder) if os.path.isfile(os.path.join(data_folder, file))]
for file in files:
    if file.endswith(".csv"):
        remove_double_quotes_from_file(file)


Double quotes removed from data/olist_order_items_dataset.csv.
Double quotes removed from data/merged_df.csv.
Double quotes removed from data/olist_sellers_dataset.csv.
Double quotes removed from data/olist_customers_dataset.csv.
Double quotes removed from data/olist_products_dataset.csv.
Double quotes removed from data/olist_geolocation_dataset.csv.
Double quotes removed from data/olist_orders_dataset.csv.
Double quotes removed from data/product_category_name_translation.csv.
Double quotes removed from data/olist_order_reviews_dataset.csv.
Double quotes removed from data/olist_order_payments_dataset.csv.


## Formatting input files for the algorithm
### Formatting Transaction database
In the **transaction database**, each line represents a transaction. Each line is composed of three sections, as follows.

1. First, the **items** contained in the transaction are listed. An item is represented by a positive integer. Each item is separated from the next item by a single space. It is assumed that all items within a same transaction are sorted according to a total order (e.g. ascending order) and that no item can appear twice within the same transaction.
2. Second, the symbol “:” appears and is followed by the **transaction utility** (an integer).
3. Third, the symbol “:” appears and is followed by the **utility of each item** in this transaction (an integer), separated by single spaces.

For example, for the previous example, the input file for the transaction is defined as follows:

1 3:6:5 1\
5:3:3\
1 2 3 4 5:25:5 10 1 6 3\
2 3 4 5:20:8 3 6 3\
1 3 4:8:5 1 2\
1 3 5:22:10 6 6\
2 3 5:9:4 2 3

Consider the first line. It means that the transaction {1, 3} has a total utility of 6 and that items 1 and 3 respectively have a utility of 5 and 1 in this transaction. The following lines follow the same format.


### Formatting Taxonomy database

In the **taxonomy database**, each line represents a category relationship. Each line is composed of two sections, as follows.

1. First, the **item**.
2. Second, the symbol “,” appears and is followed by a **generalized item**, representing the category that the item belongs.

For example, for the previous example, the input file for the **taxonomy** is defined as follows:

1,6\
2,6\
3,7\
4,8\
5,8\
6,7

Consider the first line. It means that the item 1 is belongs to the generalized item 6.



In [22]:
order_items_df = pd.read_csv("./data/olist_order_items_dataset.csv")
products_df = pd.read_csv("./data/olist_products_dataset.csv")
merged_df = pd.merge(order_items_df, products_df, on="product_id", how="inner")
print("----Dataframe shapes----")
print(f'Order items: {order_items_df.shape}')
print(f'Products: {products_df.shape}')
print(f'Merged: {merged_df.shape}')
print()

merged_df = merged_df[['order_id', 'product_id', 'price', 'product_category_name']]
merged_df.sample(10)
# print merged_df without index 

----Dataframe shapes----
Order items: (112650, 7)
Products: (32951, 9)
Merged: (112650, 15)



,order_id,product_id,price,product_category_name
40734,d994b8ab8cf4f90523706c4b34190ac4,e57eb018e37376580079825eb29a3419,80.00,watches_gifts
77912,c8cb7b0f298cd146d61e1ce533494bca,9437358367868e28d75ce47e393eef99,110.00,sports_leisure
88120,ea3da9210148306e5b1bbb843c19e65f,196c1cb4773764eca858159261a90f35,319.90,perfumery
106248,bf88003f2580db7589007ec44bc85695,fdf648fe44c2e24e9857c8d4ad5a7028,22.79,garden_tools
24246,cf650037c918409bca8e5e67ee7dcfbe,d8f254be723da6488610b42935533b05,69.00,watches_gifts
103497,aab7b04f45546e03e7c86b9e8f12368e,fa05881d3eb7ffec2322b507ed3ccc6b,19.90,housewares
15401,4644fc24ed1ffc954053a6fa1b1b825e,2028bf1b01cafb2d2b1901fca4083222,56.99,perfumery
96009,9cd3b6a2b07667e425975a219b4fc382,9eec04a029e2b9372276f3748011b493,75.25,health_beauty
4217,3d3b7c4f5d2e07cd4015a956e164a9bc,d1c427060a0f73f6b889a5c7c61f2ac4,159.99,computers_accessories
93215,cafde70fc65139700e7366af413055fe,39e2a9a2f088b50daf806b7a7e1aea82,37.95,home_construction


In [23]:
print("--------Missing values-------")
print(merged_df.isnull().sum())

print("\n-Removing all missing values-")
merged_df = merged_df.dropna()
print(merged_df.isnull().sum())


merged_df.sample(10)

--------Missing values-------
order_id                    0
product_id                  0
price                       0
product_category_name    1603
dtype: int64

-Removing all missing values-
order_id                 0
product_id               0
price                    0
product_category_name    0
dtype: int64


,order_id,product_id,price,product_category_name
81837,42c384d95181d3ee567e8639ff74fff9,f486b2e5c945f3261655ae0bcbf92855,23.90,sports_leisure
35282,137c0b15e7a9356f9ecf5da8ccc5d995,189c49b159a847050e52afe5ec8d42c5,182.00,bed_bath_table
5166,4146f35ac7a7ef4e39fe344e563c1e3b,526f833e1eb0a32ad9351145a761b9ab,129.00,watches_gifts
798,82e9ad47f8a0e270d9cbabde98cbda55,368c6c730842d78016ad823897a372db,49.90,garden_tools
71992,fbee7d34e8344a4c1775259e98e9a2a3,450c25d98ed6923154a934554475bf13,399.00,health_beauty
95059,c0624f728b31f0d7f5bafce9731a953c,2a97ccde48b4f516a7a76352d03eff88,89.90,bed_bath_table
67850,306193c0f58d9f95461c43563b65166c,4d272fe20c57c64dc74b3db191fc31ef,24.90,bed_bath_table
68522,255923114ac9ab1260799440433c6963,52b626c2fc8fffbcd01084202221bf48,99.00,garden_tools
24840,5e6e3eec3b411047e8c1c16dd5b97b02,e2e426d1332240070b0a836a854d057b,40.00,bed_bath_table
9276,48c3b4aae43bbf8e6929c347112cba89,9ecadb84c81da840dbf3564378b586e9,32.99,furniture_decor


In [24]:
encoder = LabelEncoder()
merged_df['order_id_encoded'] = encoder.fit_transform(merged_df['order_id'])
merged_df['product_id_encoded'] = encoder.fit_transform(merged_df['product_id'])
merged_df['product_id_encoded'] = merged_df['product_id_encoded'] + 1
merged_df['product_category_name_encoded'] = encoder.fit_transform(merged_df['product_category_name'])
merged_df['product_category_name_encoded'] = merged_df['product_category_name_encoded'] + max(merged_df['product_id_encoded']) + 1
merged_df['price'] = merged_df['price'].astype(int)
merged_df.sample(10)

,order_id,product_id,price,product_category_name,order_id_encoded,product_id_encoded,product_category_name_encoded
23256,449e69903d607f22bf9e35e3833f23ba,43423cdffde7fda63d0414ed38c11a73,54,watches_gifts,26033,8602,32414
106285,c9b08bd33d449366d3b89454a3071b98,829002c1d5f40820ebbc6ecd4bcdf3ab,150,home_construction,76655,16574,32390
14556,8ce27cfd1cf723e54eb8a517a0f687af,7a10781637204d8d10485c71a6108a2e,204,watches_gifts,53298,15552,32414
98785,8a40025b51fa72583cc8d0fa95439181,a1c3f3f968d87aa2c14861ee7f5a5e3b,252,sports_leisure,52347,20418,32409
49006,ba3e8a6783a3aafe0d02eeb6a415dd94,e672fbe634ad07cce9d85f412becb2c0,129,cool_stuff,70708,29113,32362
88990,5ae3075070db2a1aff1e28c35a8e5628,e61f7a619c73b49eb629e2a3ab687a7a,919,computers_accessories,34449,29076,32357
57799,7efd51cec62c158d2138c9cd808181e2,a02d0123079f4ae96001ba2010d1a2df,230,construction_tools_lights,48182,20234,32360
33740,c6094f6ab0c25eda761622ab09aa423d,fbc1488c1a1e72ba175f53ab29a248e8,118,perfumery,75268,31810,32402
14588,c1455c8a20a788e3c2d4093001d92c05,7a10781637204d8d10485c71a6108a2e,219,watches_gifts,73424,15552,32414
88238,c73f8cc3ea18990249df87fd4af96899,6ca11e79ed1acb748ccbb8e6b821967d,199,perfumery,75742,13778,32402


In [25]:
taxonomy_df = merged_df[['product_id_encoded', 'product_category_name_encoded']].drop_duplicates()
taxonomy_df = taxonomy_df.sort_values(by='product_id_encoded')
taxonomy_df.to_csv('./data/taxonomy.txt', index=False, header=False)
taxonomy_df.head(10)

,product_id_encoded,product_category_name_encoded
111565,1,32402
111787,2,32347
44201,3,32349
79011,4,32391
102982,5,32414
105784,6,32347
107382,7,32362
60429,8,32358
102030,9,32349
36133,10,32381


In [26]:
with open('./data/transaction.txt', 'w') as f:
    for order_id, group_df in merged_df.groupby('order_id'):
        product_quantities = {}
        subtotals = {}  
        
        if not group_df.empty:
            for index, row in group_df.iterrows():
                product_id = row['product_id_encoded']
                if product_id in product_quantities:
                    product_quantities[product_id] += 1
                else:
                    product_quantities[product_id] = 1

                if product_id in subtotals:
                    subtotals[product_id] += row['price']
                else:
                    subtotals[product_id] = row['price']
                
            product_ids = [str(product_id) for product_id in product_quantities.keys()]
            total_price = sum(subtotals[product_id] * quantity for product_id, quantity in product_quantities.items())

            product_totals = [str(subtotals[product_id] * quantity) for product_id, quantity in product_quantities.items()]
            formatted_line = f"{' '.join(product_ids)}:{total_price}:{' '.join(product_totals)}\n"
            f.write(formatted_line)
